In [ ]:
%cd ../..

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

from PIL import Image
import os
from models.CSRNet.CSRNet import CSRNet

import importlib
import time

In [ ]:
trained_model_path = 'D:\\Bureaublad\\save_state_ep_560_new_best_MAE_7.454.pth'  # The path to trained model file
label_factor = 100  # The label factor used to train this specific model.
dataset = 'WE_CSRNet_Meta'  # Must be the exact name of the dataset
save_results = False  # When true, save the images, GTs and predictions. A folder for this is created automatically.
set_to_eval = 'test'  # val', 'test'. Which split to test the model on. 'train' does not work!

In [ ]:
loss_fn = torch.nn.MSELoss()

In [ ]:
dataloader = importlib.import_module(f'datasets.meta.{dataset}.loading_data').loading_data
cfg_data = importlib.import_module(f'datasets.meta.{dataset}.settings').cfg_data

train_loaders, val_loaders, test_loaders, restore_transform = dataloader()
if set_to_eval == 'val':
    my_dataloaders = val_loaders
elif set_to_eval == 'test':
    my_dataloaders = test_loaders
else:
    print(f'Error: invalid set --> {set_to_eval}')

In [ ]:
save_dir = None
if save_results:
    save_folder = 'CSRNet_meta' + '_' + dataset + '_' + set_to_eval + '_' + time.strftime("%m-%d_%H-%M", time.localtime())
    save_dir = os.path.join('notebooks', save_folder)  # Manually change here is you want to save somewhere else
    os.mkdir(save_dir)

In [ ]:
model = CSRNet()

resume_state = torch.load(trained_model_path)

# new_dict = {}
# for k, v in resume_state.items():
#     k = k[4:]
#     new_dict[k] = v
# model.load_state_dict(new_dict)

model.load_state_dict(resume_state['net'])

model.cuda()
model.eval()

In [ ]:
def save_scene_graph(preds, gts, save_name):
    MAE = np.mean(np.abs(np.array(preds) - np.array(gts)))
    
#     save_path = os.path.join(save_dir, save_name)
    xs = np.arange(len(gts))
    plt.figure(figsize=(20,10))
    plt.title(f'MAE: {MAE:.3f}')
    plt.plot(xs, gts, color='green', label='GT')
    plt.plot(xs, preds, color='blue', label='Predictions')
    plt.legend()
#     plt.savefig(save_path)
    plt.show()

In [ ]:
def eval_on_scene(model, scene_dataloader):
    model.eval()
    
    with torch.no_grad():
        AEs = []  # Absolute Errors
        SEs = []  # Squared Errors
        gts = []
        preds = []

        for idx, (img, gt) in enumerate(scene_dataloader):
            img = img.cuda()
           
            den = model(img)  # Precicted density crops
            den = den.cpu()

            gt = gt.squeeze()  # Remove channel dim
            den = den.squeeze()  # Remove channel dim
            
            pred_cnt = den.sum() / label_factor
            gt_cnt = gt.sum() / cfg_data.LABEL_FACTOR
            
            AEs.append(torch.abs(pred_cnt - gt_cnt).item())
            SEs.append(torch.square(pred_cnt - gt_cnt).item())
            gts.append(gt_cnt.item())
            preds.append(pred_cnt.item())
            
        MAE = np.mean(AEs)
        MSE = np.sqrt(np.mean(SEs))

    return preds, gts, MAE, MSE

In [ ]:
MAEs = []
MSEs = []
for idx, scene_dataloader in enumerate(my_dataloaders):
    print(f'Scene {idx + 1}')

    preds, gts, MAE, MSE = eval_on_scene(model, scene_dataloader)
    print(f'    MAE/MSE: {MAE:.3f}/{MSE:.3f}')
    MAEs.append(MAE)
    MSEs.append(MSE)

    save_scene_graph(preds, gts, f'scene_{idx + 1}.jpg')
    
overal_MAE = np.mean(MAEs)
overal_MSE = np.mean(MSEs)
print(f'avg MAE/MSE: {overal_MAE:.3f}/{overal_MSE:.3f}')
        